<a href="https://www.kaggle.com/code/ibkya12/map-0-97-leashmp-competition?scriptVersionId=180590490" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 44.8 MB/s eta 0:00:00:00:0100:01


In [2]:
!pip install duckdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 50.0 MB/s eta 0:00:0000:0100:01m


## Data Preparation

The training and testing data paths are defined for the .parquet files. We use duckdb to scan search through the large training sets. Just to get started lets sample out an equal number of positive and negatives. 

This query selects an equal number of samples where binds equals 0 (non-binding) and 1 (binding), limited to 30,000 each, to avoid model bias towards a particular class.

In [3]:
import duckdb
import pandas as pd

train_path = '/kaggle/input/leash-predict-chemical-bindings/train.parquet'
test_path = '/kaggle/input/leash-predict-chemical-bindings/test.parquet'

con = duckdb.connect()

df = con.query(f"""(SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 0
                        ORDER BY random()
                        LIMIT 70000)
                        UNION ALL
                        (SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 1
                        ORDER BY random()
                        LIMIT 70000)""").df()

con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [4]:
df.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds
0,164552050,O=C(Nc1cc(C(=O)O)ccc1C(=O)O)OCC1c2ccccc2-c2ccc...,CC(C)(CN)CCS(C)(=O)=O.Cl,Nc1cc(Cl)nnc1Cl,CC(C)(CCS(C)(=O)=O)CNc1nc(Nc2cc(C(=O)N[Dy])ccc...,HSA,0
1,270432496,O=C(O)C[C@H](Cc1ccc(I)cc1)NC(=O)OCC1c2ccccc2-c...,COc1cc(F)c(Cl)cc1N,CCOC(=O)c1ncccc1N,CCOC(=O)c1ncccc1Nc1nc(Nc2cc(Cl)c(F)cc2OC)nc(N[...,HSA,0
2,236637074,O=C(O)CC(NC(=O)OCC1c2ccccc2-c2ccccc21)c1ccc(Br...,COCC1(CN)CCCCC1,Cc1nnc(N)s1,COCC1(CNc2nc(Nc3nnc(C)s3)nc(NC(CC(=O)N[Dy])c3c...,sEH,0
3,39258997,COc1cccc(NC(=O)OCC2c3ccccc3-c3ccccc32)c1C(=O)O,CC(C)(CN)C(=O)N1CCCC1,CCC1=NN(Cc2ccccc2C)C(=O)C1CCN,CCC1=NN(Cc2ccccc2C)C(=O)C1CCNc1nc(NCC(C)(C)C(=...,HSA,0
4,72993349,O=C(NC(CC1CCCCC1)C(=O)O)OCC1c2ccccc2-c2ccccc21,Nc1noc2ccc(F)cc12,Cl.NCc1noc2ccc(F)cc12,O=C(N[Dy])C(CC1CCCCC1)Nc1nc(NCc2noc3ccc(F)cc23...,HSA,0


## Feature Preprocessing

Lets grab the smiles for the fully assembled molecule `molecule_smiles` and generate ecfps for it. We could choose different radiuses or bits, but 2 and 1024 is pretty standard.

In [5]:
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import OneHotEncoder

# Convert SMILES to RDKit molecules
df['molecule'] = df['molecule_smiles'].apply(Chem.MolFromSmiles)

# Generate ECFPs
def generate_ecfp(molecule, radius=2, bits=1024):
    if molecule is None:
        return None
    return list(AllChem.GetMorganFingerprintAsBitVect(molecule, radius, nBits=bits))

df['ecfp'] = df['molecule'].apply(generate_ecfp)

## Train Model

In [6]:
# One-hot encode the protein_name
onehot_encoder = OneHotEncoder(sparse_output=False)
protein_onehot = onehot_encoder.fit_transform(df['protein_name'].values.reshape(-1, 1))

# Combine ECFPs and one-hot encoded protein_name
X = [ecfp + protein for ecfp, protein in zip(df['ecfp'].tolist(), protein_onehot.tolist())]
y = df['binds'].tolist()

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
df.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds,molecule,ecfp
0,164552050,O=C(Nc1cc(C(=O)O)ccc1C(=O)O)OCC1c2ccccc2-c2ccc...,CC(C)(CN)CCS(C)(=O)=O.Cl,Nc1cc(Cl)nnc1Cl,CC(C)(CCS(C)(=O)=O)CNc1nc(Nc2cc(C(=O)N[Dy])ccc...,HSA,0,<rdkit.Chem.rdchem.Mol object at 0x7fa2e2239700>,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, ..."
1,270432496,O=C(O)C[C@H](Cc1ccc(I)cc1)NC(=O)OCC1c2ccccc2-c...,COc1cc(F)c(Cl)cc1N,CCOC(=O)c1ncccc1N,CCOC(=O)c1ncccc1Nc1nc(Nc2cc(Cl)c(F)cc2OC)nc(N[...,HSA,0,<rdkit.Chem.rdchem.Mol object at 0x7fa2e207e180>,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,236637074,O=C(O)CC(NC(=O)OCC1c2ccccc2-c2ccccc21)c1ccc(Br...,COCC1(CN)CCCCC1,Cc1nnc(N)s1,COCC1(CNc2nc(Nc3nnc(C)s3)nc(NC(CC(=O)N[Dy])c3c...,sEH,0,<rdkit.Chem.rdchem.Mol object at 0x7fa2e207e1f0>,"[0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, ..."
3,39258997,COc1cccc(NC(=O)OCC2c3ccccc3-c3ccccc32)c1C(=O)O,CC(C)(CN)C(=O)N1CCCC1,CCC1=NN(Cc2ccccc2C)C(=O)C1CCN,CCC1=NN(Cc2ccccc2C)C(=O)C1CCNc1nc(NCC(C)(C)C(=...,HSA,0,<rdkit.Chem.rdchem.Mol object at 0x7fa2e207e260>,"[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
4,72993349,O=C(NC(CC1CCCCC1)C(=O)O)OCC1c2ccccc2-c2ccccc21,Nc1noc2ccc(F)cc12,Cl.NCc1noc2ccc(F)cc12,O=C(N[Dy])C(CC1CCCCC1)Nc1nc(NCc2noc3ccc(F)cc23...,HSA,0,<rdkit.Chem.rdchem.Mol object at 0x7fa2e207e2d0>,"[0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."


In [7]:
from xgboost import XGBClassifier

In [8]:
xgb = XGBClassifier()

xgb_model = xgb.fit(X_train , y_train)

y_pred_proba = xgb_model.predict_proba(X_test)[:, 1]  # Probability of the positive class

# Calculate the mean average precision
map_score = average_precision_score(y_test, y_pred_proba)
print(f"Mean Average Precision (mAP): {map_score:.2f}")

Mean Average Precision (mAP): 0.91


# Best Score received with CatBoostClassifier

In [9]:
from catboost import CatBoostClassifier
catb_model = CatBoostClassifier().fit(X_train, y_train)
y_pred_proba = catb_model.predict_proba(X_test)[:, 1]  # Probability of the positive class

# Calculate the mean average precision
map_score = average_precision_score(y_test, y_pred_proba)
print(f"Mean Average Precision (mAP): {map_score:.2f}")

Learning rate set to 0.012592
0:	learn: 0.6902161	total: 77.3ms	remaining: 1m 17s
1:	learn: 0.6874852	total: 93.3ms	remaining: 46.6s
2:	learn: 0.6846942	total: 106ms	remaining: 35.1s
3:	learn: 0.6802724	total: 118ms	remaining: 29.5s
4:	learn: 0.6769880	total: 131ms	remaining: 26s
5:	learn: 0.6738860	total: 142ms	remaining: 23.5s
6:	learn: 0.6706026	total: 154ms	remaining: 21.9s
7:	learn: 0.6675056	total: 168ms	remaining: 20.8s
8:	learn: 0.6646536	total: 186ms	remaining: 20.4s
9:	learn: 0.6609126	total: 200ms	remaining: 19.8s
10:	learn: 0.6577696	total: 218ms	remaining: 19.6s
11:	learn: 0.6551041	total: 233ms	remaining: 19.2s
12:	learn: 0.6530203	total: 252ms	remaining: 19.1s
13:	learn: 0.6509869	total: 264ms	remaining: 18.6s
14:	learn: 0.6486245	total: 275ms	remaining: 18.1s
15:	learn: 0.6464296	total: 287ms	remaining: 17.7s
16:	learn: 0.6443150	total: 299ms	remaining: 17.3s
17:	learn: 0.6418695	total: 310ms	remaining: 16.9s
18:	learn: 0.6391103	total: 322ms	remaining: 16.6s
19:	learn:

Look at that Average Precision score. We did amazing! 

Actually no, we just overfit. This is likely recurring theme for this competition. It is easy to predict molecules that come from the same corner of chemical space, but generalizing to new molecules is extremely difficult.

## Test Prediction

 The trained Random Forest model is then used to predict the binding probabilities. These predictions are saved to a CSV file, which serves as the submission file for the Kaggle competition.

In [ ]:
import os

# Process the test.parquet file chunk by chunk
test_file = '/kaggle/input/leash-predict-chemical-bindings/test.csv'
output_file = 'submission.csv'  # Specify the path and filename for the output file

# Read the test.parquet file into a pandas DataFrame
for df_test in pd.read_csv(test_file, chunksize=100000):

    # Generate ECFPs for the molecule_smiles
    df_test['molecule'] = df_test['molecule_smiles'].apply(Chem.MolFromSmiles)
    df_test['ecfp'] = df_test['molecule'].apply(generate_ecfp)

    # One-hot encode the protein_name
    protein_onehot = onehot_encoder.transform(df_test['protein_name'].values.reshape(-1, 1))

    # Combine ECFPs and one-hot encoded protein_name
    X_test = [ecfp + protein for ecfp, protein in zip(df_test['ecfp'].tolist(), protein_onehot.tolist())]

    # Predict the probabilities
    probabilities = catb_model.predict_proba(X_test)[:, 1]

    # Create a DataFrame with 'id' and 'probability' columns
    output_df = pd.DataFrame({'id': df_test['id'], 'binds': probabilities})

    # Save the output DataFrame to a CSV file
    output_df.to_csv(output_file, index=False, mode='a', header=not os.path.exists(output_file))
